In [ ]:
import cv2 as cv
import scipy.io as sio
from tqdm import tqdm
from console_progressbar import ProgressBar

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import keras
import pandas as pd
from PIL import Image
import cv2 as cv
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import SGD, Adam
from keras.layers.pooling import MaxPooling2D,AveragePooling2D

In [3]:
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, num_classes):

    x = base_model.output 

    x = AveragePooling2D((5, 5), name='avg_pool')(x)
    x = Flatten()(x)
    x = Dropout(dropout)(x)
    predictions = Dense(num_classes, activation='softmax', name='finalfc')(x)
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

In [4]:
HEIGHT = 299
WIDTH = 299

input_shape=(HEIGHT, WIDTH, 3)

dropout = 0.7

base_model = EfficientNetB0(weights='imagenet',include_top=False,input_shape=(HEIGHT, WIDTH, 3))

model = build_finetune_model(base_model, dropout=dropout, num_classes=196)

model.load_weights('keras_swa.model')


2022-01-05 09:41:43.383800: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def save_test_data(fnames, bboxes):
    src_folder = 'classifier_evaluation'
    dst_folder = ''
    num_samples = len(fnames)

    pb = ProgressBar(total=100, prefix='Save test data', suffix='', decimals=3, length=50, fill='=')

    for i in range(num_samples):
        fname = fnames[i]
        (x1, y1, x2, y2) = bboxes[i]
        src_path = src_folder + str(fname)
        src_image = cv.imread(src_path)
        height, width = src_image.shape[:2]
        # margins of 16 pixels
        margin = 16
        x1 = max(0, x1 - margin)
        y1 = max(0, y1 - margin)
        x2 = min(x2 + margin, width)
        y2 = min(y2 + margin, height)
        # print(fname)
        pb.print_progress_bar((i + 1) * 100 / num_samples)

        dst_path = os.path.join(dst_folder, fname)
        crop_image = src_image[y1:y2, x1:x2]
        dst_img = cv.resize(src=crop_image, dsize=(img_height, img_width))
        cv.imwrite(dst_path, dst_img)


In [7]:
all_preds, file = [], []

tmp = []
for filename in tqdm(os.listdir('cars_test')):
    tmp += [filename]

tmp.sort()
num_samples =len(tmp)

for i in tqdm(range(num_samples)):
    filename = os.path.join('cars_test',tmp[i])
    bgr_img = cv.imread(filename)
    rgb_img = cv.resize(cv.cvtColor(bgr_img, cv.COLOR_BGR2RGB)/255,(299,299))
    rgb_img = np.expand_dims(rgb_img, 0)
    preds = model.predict(rgb_img)
    class_id = np.argmax(preds)
    all_preds.append(class_id+1)
    file += [tmp[i]]


 
result = {'filename' : file,
          'pred_label' : all_preds}
    



100%|██████████| 8041/8041 [51:16<00:00,  2.61it/s]     


In [8]:
result_df = pd.DataFrame(result)
print(result_df)

       filename  pred_label
0     00001.jpg         181
1     00002.jpg          93
2     00003.jpg         145
3     00004.jpg         187
4     00005.jpg         185
...         ...         ...
8036  08037.jpg          63
8037  08038.jpg          16
8038  08039.jpg          16
8039  08040.jpg          38
8040  08041.jpg          32

[8041 rows x 2 columns]


In [9]:
labels = sio.loadmat('devkit/cars_test_annos_withlabels.mat')
actual_label = np.array(labels['annotations']['class'],dtype=np.int)
actual_label = actual_label.squeeze()
all_preds = np.array(all_preds)
actual_label = actual_label


In [10]:
count = np.sum(actual_label == all_preds)
print('accuracy = ', count/len(actual_label))

accuracy =  0.8883223479666709
